# Download GDEL

In [121]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])] # mengambil file x yang string are digits 4 baris pertama
file_list2 = file_list[0:10]   # mengambil file di list GDELT yang 2 baris pertama
print(file_list2)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'D:/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list2[infilecounter:]:
    print(compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
        print('done')

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')
    print('done')

    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter

        # open the infile and outfile
        with open(infile_name, encoding='utf8', mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

['20181101.export.CSV.zip']
20181101.export.CSV.zip
extracting,
done
parsing,
done


# PICKING  DATA

In [122]:
import glob
import pandas as pd

local_path = 'D:/GDELT/'
fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel('CSV.header.fieldids.xlsx', sheetname='Sheet1',
                         index_col='Column ID', usecols=1)['Field Name']
# print (colnames)
# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print (files)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))
    break

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)

['D:/GDELT/country\\ID0000.tsv']


# List Data dalam Folder

In [124]:
from os import listdir
from os.path import isfile, join
folder_id = 'D:/GDELT/country/'
files = [f for f in listdir(folder_id) if isfile(join(folder_id, f))]
print(files)

['ID0000.tsv', 'ID0001.tsv', 'ID0002.tsv', 'ID0003.tsv', 'ID0004.tsv', 'ID0005.tsv', 'ID0006.tsv', 'ID0007.tsv', 'ID0008.tsv', 'ID0009.tsv']


In [125]:
# Open first file for sanity check
df_awal = pd.read_csv(folder_id + files[9],sep="\t",encoding='latin-1') # melihat sampel isi data
df_awal.head()

,796649358,20171023,201710,2017,2017.8027,BUS,CORPORATION,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,ID.2,4.1,"Manila, Manila, Philippines.1",RP.1,RPD9.1,14.6042.1,120.982.1,-2437894.1,20181023,https://www.dealstreetasia.com/stories/chinese-fintech-lenders-philippines-109510/
0,796650128,20181023,201810,2018,2018.8027,AFR,AFRICA,AFR,NaN,NaN,...,NaN,4,"Komodo, Indonesia (general), Indonesia",ID,ID00,-8.58333,119.500,-2683074,20181023,https://www.escape.com.au/news/better-than-bal...
1,796650297,20181023,201810,2018,2018.8027,BUS,INDUSTRY,NaN,NaN,NaN,...,NaN,4,"Gudang, Jawa Tengah, Indonesia",ID,ID07,-6.71010,110.947,10171924,20181023,https://www.dealstreetasia.com/stories/indones...
2,796650336,20181023,201810,2018,2018.8027,BUS,ENTREPRENEUR,NaN,NaN,NaN,...,10171924,4,"Gudang, Jawa Tengah, Indonesia",ID,ID07,-6.71010,110.947,10171924,20181023,https://www.dealstreetasia.com/stories/indones...
3,796651934,20181023,201810,2018,2018.8027,IDN,INDONESIA,IDN,NaN,NaN,...,NaN,4,"Gudang, Jawa Tengah, Indonesia",ID,ID07,-6.71010,110.947,10171924,20181023,https://www.dealstreetasia.com/stories/indones...
4,796651935,20181023,201810,2018,2018.8027,IDN,INDONESIA,IDN,NaN,NaN,...,10171924,4,"Gudang, Jawa Tengah, Indonesia",ID,ID07,-6.71010,110.947,10171924,20181023,https://www.dealstreetasia.com/stories/indones...


# Simpan ke MySQL

Setetelah di List filenya, kemudian semua file yang ada di D:/GDELT/country/ dimasukkan ke MySQL dengan nama tabel per setiap file

In [126]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = '3306'
username = 'root'
password = ''
database = 'gdelt_content'

# Create Connection to database
engine = create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database)
'''engine = create_engine('mysql+pymysql://root: @localhost:3306/gdelt_content_id')'''

def run(sql):
    df = pd.read_sql_query(sql, engine)
    return df

In [130]:
# Read header / column names
colnames = list(pd.read_excel('CSV.header.fieldids.xlsx', sheet_name='CSV.header.dailyupdates'))

for berkas in files:
    print('Extracting ' + berkas)
    df_satuan = pd.DataFrame()
    
    # Important: If your ID0000.tsv contains header, you can delete the header
    df_satuan = pd.read_csv(folder_id + berkas, sep="\t", names= colnames, encoding='latin-1')
    df_satuan.to_sql(name = berkas, con = engine, if_exists = 'append', index = False) # Memasukkan data ke tabel sesuai dengan file

Extracting ID0000.tsv
Extracting ID0001.tsv
Extracting ID0002.tsv
Extracting ID0003.tsv
Extracting ID0004.tsv
Extracting ID0005.tsv
Extracting ID0006.tsv
Extracting ID0007.tsv
Extracting ID0008.tsv
Extracting ID0009.tsv
